In [1]:
import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
import seaborn as sns
import shutil
from imageio import imread
import imageio
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from keras.applications.resnet import ResNet50, ResNet101, ResNet152
import datetime
from keras.callbacks import TensorBoard
import cv2
from skimage.transform import resize
import PIL
from numpy import save,load
from keras.optimizers import Adam,SGD
%env SM_FRAMEWORK=tf.keras

import tensorflow as tf

from keras import backend as K
import tensorflow_addons as tfa


from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Input, Add, Dropout,Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, concatenate,Conv2DTranspose,GlobalMaxPool2D,GlobalAveragePooling2D,UpSampling2D
import gc

!pip install segmentation-models
from segmentation_models import Unet,PSPNet,FPN,Linknet


PATH            = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/'
TRAIN_PATH      = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train/'
TRAIN_MASK_PATH = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train_mask/'
TRAIN_ELA_PATH  = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train_ela/'
TEST_PATH       = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/test/'
TEST_ELA_PATH   = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/test_ela/'

CKPT_PATH       = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/model/'

env: SM_FRAMEWORK=tf.keras
     |████████████████████████████████| 51kB 4.0MB/s 
Segmentation Models: using `tf.keras` framework.


In [2]:
! /opt/bin/nvidia-smi 

Tue Nov 24 13:23:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Load Data

In [3]:
X_train = load(PATH+'npy/X_train.npy')
y_train  = load(PATH+'npy/Y_train.npy')

X_valid = load(PATH+'npy/X_valid.npy')
y_valid  = load(PATH+'npy/Y_valid.npy')

In [4]:
# X_train = np.concatenate([X_train1,X_train2])
# y_train = np.concatenate([y_train1,y_train2])

#Seg Models

In [5]:
# Models
# + Unet
# + FPN
# + Linknet
# + PSPNet

# | Type         | Names                                                               |
# |--------------|---------------------------------------------------------------------|
# | VGG          | 'vgg16' 'vgg19'                                                     |
# | ResNet       | 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152'            |
# | SE-ResNet    | 'seresnet18' 'seresnet34' 'seresnet50' 'seresnet101' 'seresnet152'  |
# | ResNeXt      | 'resnext50' 'resnext101'                                            |
# | SE-ResNeXt   | 'seresnext50' 'seresnext101'                                        |
# | SENet154     | 'senet154'                                                          |
# | DenseNet     | 'densenet121' 'densenet169' 'densenet201'                           |
# | Inception    | 'inceptionv3' 'inceptionresnetv2'                                   |
# | MobileNet    | 'mobilenet' 'mobilenetv2'                                           |
# | EfficientNet | 'efficientnetb0' 'efficientnetb1' 'efficientnetb2' 'efficientnetb3' |

In [6]:
# Dice_Coeff or F1 score
def metric(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

model_name = 'Unet'
backbone_name = 'efficientnetb7'
model = Unet(backbone_name, classes=1, activation='sigmoid')
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=metric)

#print(model.summary(line_length=120))

258441216/258434480 [==============================] - 4s 0us/step


In [ ]:
 model.fit(X_train,Y_trian, steps_per_epoch=(spe), epochs=nb_epochs,validation_data=(X_valid, y_valid),callbacks=[save,lr_schedule,reduce_lr])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import time
                  
t= time.strftime("%Y%m%d-%H%M%S", time.localtime())
### MODELCHECKPOINT CALLBACK
save = ModelCheckpoint(CKPT_PATH+'{}_{}_{}_model.h5'.format(t,model_name,backbone_name), verbose=1, save_best_only=True, save_weights_only=True)


### REDUCES LR WHEN METRTIC IS NOT IMPROVING
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta =1e-4, min_lr=0.00001, mode='min')

### LEARNING RATE SCHEDULER
def scheduler(epoch):

  if epoch < 10:
     return 0.001
  else:
     return float(0.001 * tf.math.exp(0.1 * (10 - epoch)))
lr_schedule = LearningRateScheduler(scheduler) 

### EARLY STOPPING
early_stopping = EarlyStopping(patience=10, verbose=1)

# Data Generator and augmentation
data_gen_args = dict(horizontal_flip=True,vertical_flip=True)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 62
bs = 4
nb_epochs = 100

image_generator = image_datagen.flow(X_train, seed=seed, batch_size=bs, shuffle=True)
mask_generator = mask_datagen.flow(y_train, seed=seed, batch_size=bs, shuffle=True)

# Just zip the two generators to get a generator that provides augmented images and masks at the same time
train_generator = zip(image_generator, mask_generator)

In [ ]:
spe = len(X_train) // bs
del X_train
del y_train
del image_generator
del mask_generator

In [ ]:
results = model.fit_generator(train_generator, steps_per_epoch=(spe), epochs=nb_epochs,validation_data=(X_valid, y_valid),callbacks=[save,lr_schedule,reduce_lr])
model.save_weights(CKPT_PATH+'{}_{}_{}_model.h5'.format(t,model_name,backbone_name))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
predicted = model.predict(X_valid)

In [ ]:
def plot_predicted_images(index):
    pred = np.squeeze(predicted[index])
    plt.imsave('pred_mask.png',pred)
    im_gray = cv2.imread('pred_mask.png', cv2.IMREAD_GRAYSCALE)
    (thresh, im_bw) = cv2.threshold(im_gray, 220, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    fig = plt.figure(figsize=(15,15))
    ax1 = fig.add_subplot(331)
    ax2 = fig.add_subplot(332)
    ax3 = fig.add_subplot(333)   
    ax1.set_title("pristine image")
    ax2.set_title("original mask")
    ax3.set_title("predicted binary mask")
    ax1.imshow(X_valid[index])
    ax2.imshow(np.squeeze(y_valid[index]))
    ax3.imshow(im_bw)

In [ ]:
import random
for i in range(1,11):
    x = random.randint(1,200)
    plot_predicted_images(x)